# Regression Model to determine influence of artist's gender on the topics generated by BERTopic

In [1]:
from google.colab import drive
import os

  # gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

  # # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# To run from the common drive:
dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from bertopic import BERTopic
# from umap import UMAP

# set path of the file
csv_file_path = '/results/topics_for_bias_analysis.csv'
df = pd.read_csv(dataset_path + csv_file_path)

In [3]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,genre,artist,year,views,features,lyrics,language,gender,cleaned_lyrics,has_lyrics,topic,topic_prob,topic_label
0,0,0,0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",en,Male,"\nKilla Cam, Killa Cam, Cam\nKilla Cam, Killa ...",True,7,0.609817,mi_dem_yuh
1,1,1,1,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,en,Male,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",True,0,0.675691,nigga_niggas_bitch
2,2,2,2,Family Ties,rap,Cam'ron,2004,41960,"{""Cam\\'ron"",""Lady Wray""}","[Verse 1: Cam'ron]\nKilla, Dipset\nMan I spit ...",en,Male,"\nKilla, Dipset\nMan I spit that pimp talk, yo...",True,434,0.652398,bicyclette_ma bicyclette_bicyclette ma
3,3,3,3,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,en,Male,\nAy yo you wonder who I are\nI guzzle up at t...,True,434,0.742198,bicyclette_ma bicyclette_bicyclette ma
4,4,4,4,Lord You Know,rap,Cam'ron,2004,11882,"{""Cam\\'ron"",""Juelz Santana"",Jaheim}","[Chorus: Jaheim]\nNow Lord you know, just how ...",en,Male,"\nNow Lord you know, just how hard I try\nTo l...",True,398,0.737625,dog status_movin rank_status im


In [18]:
dataset = df[['genre', 'year', 'gender', 'topic', 'topic_prob']]

In [19]:
dataset.describe()

,year,topic,topic_prob
count,537553.000000,537553.000000,537553.000000
mean,2002.695682,165.225821,0.684348
std,23.488690,156.126712,0.093572
min,1.000000,-1.000000,0.050249
25%,1996.000000,28.000000,0.630626
50%,2008.000000,110.000000,0.693565
75%,2015.000000,269.000000,0.746582
max,2023.000000,541.000000,0.995553


In [20]:
# Converting year to decade
dataset['year'].unique()

array([2004, 1998, 2002, 2009, 2000, 2006, 2010, 2011, 2003, 2014, 2007,
       2005, 1999, 2013, 2015, 2008, 2001, 2020, 2012, 2016, 2019, 1995,
       1994, 1997, 1993, 2017, 1992, 1996, 1988, 1990, 1991, 1960, 2018,
       1989, 1965, 1975, 1962, 1974, 1964, 1963, 1976, 1966, 1967, 1985,
       1979, 1986, 1961, 1981, 1980, 1978, 1969, 1970, 1971, 1968, 1973,
       1983, 1987, 1972, 1977, 1982, 2022, 1984, 2021, 1956,    1, 1950,
       1953, 1951, 1949, 1958, 1955, 1957, 1946, 1959, 1954, 1952, 1945,
       1948, 1947, 1939, 1871, 1876, 1865, 1415, 2023, 1770, 1944, 1855,
       1936,   15, 1814, 1759, 1785, 1788, 1789, 1791, 1782, 1929, 1938,
       1926, 1931, 1776, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808,
       1940, 1941, 1932, 1933, 1943, 1942,    2,  699, 1861, 1937, 1935,
       1934, 1862, 1918, 1919, 1917, 1930, 1517, 1790, 1787, 1928, 1927,
       1895, 1907, 1911, 1925, 1906, 1904, 1923,   79, 1864, 1847, 1921,
       1741, 1920, 1905, 1870, 1924, 1922, 1739, 17

In [21]:
dataset['year'].max()

2023

In [22]:
# Number of songs with year < 1950 - a few of these are from the 18th and 19th centuries but there are also some mistagged songs

print(len(dataset[dataset['year'] < 1950])) # => 2856, only 0.5% of the data so these are ignored
dataset = dataset[dataset['year'] >= 1950]

def get_decade(year):
  if year >= 2000 and year < 2010:
    decade = '2000s'
  else:
    decade = str(year//100) + str(year%100)[0] + '0s'
  return decade

dataset['decade'] = dataset['year'].apply(get_decade)

2856


In [23]:
# Number of outliers

len(df[df['topic'] == -1])

8598

In [24]:
dataset = dataset[dataset['topic'] != -1]
dataset = dataset[dataset['genre'] != 'misc']

In [25]:
top10topics = dataset['topic'].value_counts().nlargest(10).to_dict()

In [33]:
dataset['decade'] = dataset['year'].apply(get_decade)

In [34]:
mask = dataset['topic'].isin(top10topics)
top10 = dataset[mask]

In [35]:
top10

,genre,year,gender,topic,topic_prob,decade
1,rap,2004,Male,0,0.675691,2000s
5,rap,2002,Male,0,0.776709,2000s
6,rap,2004,Male,0,0.778837,2000s
7,rap,2004,Male,0,0.740868,2000s
11,rap,2006,Male,0,0.706905,2000s
...,...,...,...,...,...,...
537546,pop,2020,Male,1,0.795815,2020s
537547,pop,2020,Male,1,0.770242,2020s
537548,pop,2020,Male,1,0.745915,2020s
537549,pop,2022,Female,3,0.539879,2020s


In [36]:
top10_X = top10[['genre', 'decade', 'gender']]
top10_y = top10['topic']

In [37]:
top10

,genre,year,gender,topic,topic_prob,decade
1,rap,2004,Male,0,0.675691,2000s
5,rap,2002,Male,0,0.776709,2000s
6,rap,2004,Male,0,0.778837,2000s
7,rap,2004,Male,0,0.740868,2000s
11,rap,2006,Male,0,0.706905,2000s
...,...,...,...,...,...,...
537546,pop,2020,Male,1,0.795815,2020s
537547,pop,2020,Male,1,0.770242,2020s
537548,pop,2020,Male,1,0.745915,2020s
537549,pop,2022,Female,3,0.539879,2020s


In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(top10_X, top10_y, test_size=0.2, random_state=42)

In [40]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# le = LabelEncoder()
cat = OneHotEncoder()
# cat_dec = OneHotEncoder()

preprocessor = ColumnTransformer(transformers=[
    ('cat', cat, ['genre', 'decade', 'gender'])
])
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [43]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

In [44]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [45]:
y_hat = model.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score

accuracy_score(y_hat, y_test)

0.4733125223888235

In [48]:
parameters = model.get_params()
parameters

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'multinomial',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [51]:
coefficients = model.coef_
coefficients

array([[-1.71715661, -0.09696799,  3.18098743, -0.0107723 , -1.34251477,
        -0.78860406, -1.12154135, -0.97559374, -0.30520624,  0.78269545,
         0.89399074,  1.09813872,  0.42969624, -0.60818451,  0.62176027],
       [ 0.87943062,  0.51778353, -1.78195091, -0.07192738,  0.45188963,
         0.50256395,  0.02590425,  0.03214936,  0.12627201,  0.1705565 ,
        -0.12948815, -0.39305704, -0.3396754 ,  0.07591054, -0.08068504],
       [-0.31665766, -0.30298493,  0.0682823 ,  1.01471348, -0.47402072,
         0.14115763, -0.07204308, -0.24986   , -0.19344839, -0.20056877,
         0.09842033,  0.18957012,  0.27610463,  0.08352429, -0.09419182],
       [-0.40022278,  0.09341652, -0.22212022,  0.39430584,  0.12346635,
         1.15036862,  0.52416218,  0.36306925,  0.10769378, -0.42876003,
        -0.51664072, -0.52968905, -0.68135832,  0.12152089, -0.13267519],
       [ 0.2694943 ,  0.09666278, -0.68474072, -0.43664885,  0.75341663,
        -0.98966049, -0.14896448,  0.17463197, 